# Read Preprocessed Time Series Data

In [2]:
# Please run ts_regularization.ipynb to get required data

import pandas as pd
import numpy as np

X_train = pd.read_pickle("X_train_znorm_padded_interpolated.pickle")
X_test = pd.read_pickle("X_test_znorm_padded_interpolated.pickle")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [3]:
y_train = np.where(y_train=="UGE", 1, 0)
y_test = np.where(y_test=="UGE", 1, 0)

In [4]:
X_train.head()

,0,1,2,3,4,5,6,7
0,0 -0.079347 1 -0.057855 2 -0.06...,0 -0.216211 1 -0.213989 2 -0.21...,0 -0.099801 1 -0.097689 2 -0.09...,0 -0.036601 1 -0.032656 2 -0.03...,0 -0.643445 1 -0.444213 2 -0.63...,0 -0.478618 1 -0.205830 2 -0.37...,0 -0.646254 1 0.434226 2 -0.64...,0 5.070211 1 2.515122 2 -0.33...
1,0 -0.222481 1 -0.222481 2 -0.22...,0 -0.256157 1 -0.256157 2 -0.25...,0 -0.041321 1 -0.041321 2 -0.04...,0 -0.040765 1 -0.040765 2 -0.04...,0 -0.359279 1 -0.359279 2 -0.35...,0 0.260369 1 0.260369 2 0.39...,0 -0.272245 1 -0.272245 2 -0.27...,0 -0.146473 1 -0.146473 2 -0.14...
2,0 -0.127718 1 -0.092490 2 -0.16...,0 -0.110700 1 -0.088013 2 -0.12...,0 -0.119810 1 -0.106750 2 -0.12...,0 -0.095365 1 -0.081636 2 -0.10...,0 -0.329829 1 0.088769 2 -0.46...,0 -0.247747 1 0.573471 2 -0.55...,0 -0.358458 1 -0.278369 2 -0.35...,0 -0.265594 1 -0.265594 2 6.82...
3,0 -0.365931 1 0.008711 2 0.47...,0 -0.258248 1 0.050251 2 -0.23...,0 -0.073856 1 -0.065194 2 0.08...,0 0.452482 1 -0.046502 2 0.05...,0 -0.502186 1 1.312936 2 -0.44...,0 -0.615960 1 0.219886 2 0.84...,0 -0.342548 1 0.266786 2 -0.34...,0 3.705535 1 -0.274311 2 -0.27...
4,0 -0.158076 1 -0.118848 2 -0.14...,0 -0.101253 1 -0.078526 2 -0.08...,0 -0.081304 1 -0.070671 2 -0.07...,0 -0.211056 1 0.237259 2 -0.21...,0 -0.230478 1 -0.203869 2 0.35...,0 -0.113308 1 0.050849 2 -0.12...,0 -0.287139 1 -0.287139 2 -0.28...,0 -0.367491 1 -0.367491 2 -0.36...


# Create Train/Test Sequences

In [7]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics import Accuracy

In [8]:
train_sequences = []

for user_id, temp_df in X_train.groupby(level=0):
    seq_df = pd.DataFrame()
    for i in range(8):
        seq_df[f"indicator{i+1}"] = temp_df.iloc[:, i].values[0]
    train_sequences.append((seq_df, y_train[user_id]))

In [9]:
test_sequences = []

for user_id, temp_df in X_test.groupby(level=0):
    seq_df = pd.DataFrame()
    for i in range(8):
        seq_df[f"indicator{i+1}"] = temp_df.iloc[:, i].values[0]
    test_sequences.append((seq_df, y_test[user_id]))

In [10]:
train_sequences[0]

(      indicator1  indicator2  indicator3  indicator4  indicator5  indicator6  \
 0      -0.079347   -0.216211   -0.099801   -0.036601   -0.643445   -0.478618   
 1      -0.057855   -0.213989   -0.097689   -0.032656   -0.444213   -0.205830   
 2      -0.065915   -0.215470   -0.099801   -0.034409   -0.635475   -0.373700   
 3      -0.044423   -0.214730   -0.092056   -0.030902   -0.595629   -0.205830   
 4      -0.079347   -0.216211   -0.099801   -0.036601   -0.643445   -0.478618   
 ...          ...         ...         ...         ...         ...         ...   
 2011    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2012    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2013    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2014    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2015    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 
       indicator7  indicat

In [11]:
test_sequences[0]

(      indicator1  indicator2  indicator3  indicator4  indicator5  indicator6  \
 0      -0.003523   -0.365918   -0.244414   -0.054658   -0.197772   -0.106472   
 1      -0.398581   -0.680598   -0.244414    0.009228   -0.476931    1.407609   
 2      -0.158724   -0.432296   -0.240645   -0.062643   -0.395331   -0.151668   
 3       0.034102   -0.311488   -0.237968   -0.052722   -0.369563   -0.016079   
 4       0.069092   -0.285847   -0.207810   -0.052722   -0.408215    0.051716   
 ...          ...         ...         ...         ...         ...         ...   
 2011    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2012    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2013    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2014    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 2015    0.000001    0.000001    0.000001    0.000001    0.000001    0.000001   
 
       indicator7  indicat

In [12]:
len(train_sequences), len(test_sequences)

(400, 100)

# Initialize Train/Test Sequence Data Loaders

In [13]:
class TSDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        return dict(
            sequence = torch.Tensor(sequence.to_numpy()),
            label = torch.tensor(label).long()
        )

In [14]:
class TSDataModule(pl.LightningDataModule):
    def __init__(self, train_sequences, test_sequences, batch_size):
        super().__init__()
        self.train_sequences = train_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size
        
    def setup(self, stage=None):
        self.train_dataset = TSDataset(self.train_sequences)
        self.test_dataset = TSDataset(self.test_sequences)
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size = self.batch_size,
            shuffle=True,
            num_workers=8
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size,
            shuffle=False,
            num_workers=8
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size,
            shuffle=False,
            num_workers=8
        )

# Define RNN Architecture

In [15]:
N_EPOCHS = 250
BATCH_SIZE = 10

data_module = TSDataModule(train_sequences, test_sequences, BATCH_SIZE)

In [16]:
class SequenceModel(nn.Module):
    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        super().__init__()
    
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.75
        )
    
        self.classifier = nn.Linear(n_hidden, n_classes)
    
    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        
        out = hidden[-1]
        return self.classifier(out)

In [17]:
class TSPredictor(pl.LightningModule):
    def __init__(self, n_features: int, n_classes: int):
        super().__init__()
        self.model = SequenceModel(n_features, n_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.acc = Accuracy()
        
    def forward(self, x, labels=None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output
    
    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = self.acc(predictions, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_acc", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}
    
    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = self.acc(predictions, labels)
        
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_acc", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}
    
    def testing_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = self.acc(predictions, labels)
        
        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.log("test_acc", step_accuracy, prog_bar=True, logger=True)
        return {"loss": loss, "accuracy": step_accuracy}
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.0001)

In [18]:
model = TSPredictor(n_features=8, n_classes=2)

# Run Model with Best Checkpoint Callback and TensorBoard Logger

In [19]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_acc",
    mode="max"
)

logger = TensorBoardLogger("lightning_logs", name="home_network_user_classifier")

trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    accelerator="gpu",
    devices=1,
    log_every_n_steps=30
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(model, data_module)

Missing logger folder: lightning_logs/home_network_user_classifier
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | SequenceModel    | 1.3 M 
1 | criterion | CrossEntropyLoss | 0     
2 | acc       | Accuracy         | 0     
-----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.302     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 40: 'val_acc' reached 0.50000 (best 0.50000), saving model to '/notebooks/checkpoints/best-checkpoint.ckpt' as top 1


...

Epoch 247, global step 9920: 'val_acc' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 248, global step 9960: 'val_acc' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 249, global step 10000: 'val_acc' was not in top 1
`Trainer.fit` stopped: `max_epochs=250` reached.
